In [108]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [109]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [110]:
train_csv = drive.CreateFile({'id':'1B109R7m7CpVVDuA2klr-xcX5Qa2k9ncX'})
test_csv = drive.CreateFile({'id':'1kb_5tpvLJmrTPg7vN3CyVR0nUIiD2Ghx'})

In [111]:
import pandas as pd

train_csv.GetContentFile('train.csv')  
text = pd.read_csv('train.csv')
test_csv.GetContentFile('test.csv')  
test_text = pd.read_csv('test.csv')

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

In [112]:
text.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [113]:
test_text.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


### Modelo 1

In [114]:
from sklearn.model_selection import train_test_split 
import numpy as np

x_train, x_test, y_train, y_test = \
train_test_split(text['text'], text['target'], test_size = 0.25, random_state = 123)

In [120]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(x_train[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[-0.12947434, -0.9392339 , -0.77080125, -0.21456313,  0.5920508 ,
        -1.3340634 , -0.4480354 , -0.34861115, -0.39928764,  1.0191009 ,
        -1.0532486 ,  0.99523056, -0.3328474 , -0.19521488, -1.2477547 ,
         1.549057  , -0.9475413 , -0.9282966 , -1.024526  ,  0.7211244 ]],
      dtype=float32)>

In [121]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_25 (KerasLayer)  (None, 20)                400020    
_________________________________________________________________
dense_50 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [122]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [123]:
history = model.fit(x_train,
                    y_train,
                    epochs=70,
                    batch_size=512,
                    validation_data=(x_test, y_test),
                    verbose=1)

Epoch 1/70
12/12 [==============================] - 0s 23ms/step - loss: 1.0238 - accuracy: 0.4954 - val_loss: 0.9106 - val_accuracy: 0.5252
Epoch 2/70
12/12 [==============================] - 0s 14ms/step - loss: 0.8164 - accuracy: 0.5572 - val_loss: 0.7509 - val_accuracy: 0.5982
Epoch 3/70
12/12 [==============================] - 0s 15ms/step - loss: 0.6893 - accuracy: 0.6499 - val_loss: 0.6608 - val_accuracy: 0.6660
Epoch 4/70
12/12 [==============================] - 0s 15ms/step - loss: 0.6204 - accuracy: 0.6949 - val_loss: 0.6137 - val_accuracy: 0.6765
Epoch 5/70
12/12 [==============================] - 0s 14ms/step - loss: 0.5809 - accuracy: 0.7120 - val_loss: 0.5869 - val_accuracy: 0.6933
Epoch 6/70
12/12 [==============================] - 0s 13ms/step - loss: 0.5562 - accuracy: 0.7208 - val_loss: 0.5702 - val_accuracy: 0.7022
Epoch 7/70
12/12 [==============================] - 0s 13ms/step - loss: 0.5388 - accuracy: 0.7304 - val_loss: 0.5587 - val_accuracy: 0.7106
Epoch 8/70
12

In [124]:
results = model.evaluate(x_test, y_test)

print(results)

60/60 [==============================] - 0s 2ms/step - loss: 0.6102 - accuracy: 0.7904
[0.6102109551429749, 0.7904411554336548]


In [73]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(x_train[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[-0.12947434, -0.9392339 , -0.77080125, -0.21456313,  0.5920508 ,
        -1.3340634 , -0.4480354 , -0.34861115, -0.39928764,  1.0191009 ,
        -1.0532486 ,  0.99523056, -0.3328474 , -0.19521488, -1.2477547 ,
         1.549057  , -0.9475413 , -0.9282966 , -1.024526  ,  0.7211244 ]],
      dtype=float32)>

In [90]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_14 (KerasLayer)  (None, 20)                400020    
_________________________________________________________________
dense_34 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_35 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 17        
Total params: 400,645
Trainable params: 400,645
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [92]:
history = model.fit(x_train,
                    y_train,
                    epochs=100,
                    batch_size=512,
                    validation_data=(x_test, y_test),
                    verbose=1,
                    callbacks = [callback])

Epoch 1/100
12/12 [==============================] - 0s 22ms/step - loss: 0.8617 - accuracy: 0.4311 - val_loss: 0.8532 - val_accuracy: 0.4249
Epoch 2/100
12/12 [==============================] - 0s 14ms/step - loss: 0.8350 - accuracy: 0.4339 - val_loss: 0.8194 - val_accuracy: 0.4433
Epoch 3/100
12/12 [==============================] - 0s 14ms/step - loss: 0.7893 - accuracy: 0.4985 - val_loss: 0.7650 - val_accuracy: 0.5525
Epoch 4/100
12/12 [==============================] - 0s 14ms/step - loss: 0.7246 - accuracy: 0.6931 - val_loss: 0.7099 - val_accuracy: 0.7017
Epoch 5/100
12/12 [==============================] - 0s 14ms/step - loss: 0.6767 - accuracy: 0.7387 - val_loss: 0.6804 - val_accuracy: 0.7048
Epoch 6/100
12/12 [==============================] - 0s 15ms/step - loss: 0.6513 - accuracy: 0.7569 - val_loss: 0.6660 - val_accuracy: 0.7274
Epoch 7/100
12/12 [==============================] - 0s 14ms/step - loss: 0.6351 - accuracy: 0.7979 - val_loss: 0.6570 - val_accuracy: 0.7505
Epoch 

In [80]:
results = model.evaluate(x_test, y_test)

print(results)

60/60 [==============================] - 0s 2ms/step - loss: 0.6273 - accuracy: 0.7868
[0.6273136138916016, 0.7867646813392639]


### Modelo 2

In [26]:
x_train, x_test, y_train, y_test = \
train_test_split(text['text'], text['target'], test_size = 0.25, random_state = 123)

In [128]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", output_shape=[50],
                           input_shape=[], dtype=tf.string)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_27 (KerasLayer)  (None, 50)                48190600  
_________________________________________________________________
dense_54 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 11        
Total params: 48,191,121
Trainable params: 521
Non-trainable params: 48,190,600
_________________________________________________________________


In [129]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [130]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)

history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_test, y_test),
                    verbose=1,
                    callbacks = [callback])

Epoch 1/40
12/12 [==============================] - 1s 94ms/step - loss: 0.7532 - accuracy: 0.5686 - val_loss: 0.7556 - val_accuracy: 0.5762
Epoch 2/40
12/12 [==============================] - 1s 87ms/step - loss: 0.7510 - accuracy: 0.5684 - val_loss: 0.7534 - val_accuracy: 0.5762
Epoch 3/40
12/12 [==============================] - 1s 86ms/step - loss: 0.7490 - accuracy: 0.5684 - val_loss: 0.7513 - val_accuracy: 0.5762
Epoch 4/40
12/12 [==============================] - 1s 88ms/step - loss: 0.7469 - accuracy: 0.5684 - val_loss: 0.7492 - val_accuracy: 0.5762
Epoch 5/40
12/12 [==============================] - 1s 86ms/step - loss: 0.7449 - accuracy: 0.5684 - val_loss: 0.7472 - val_accuracy: 0.5762
Epoch 6/40
12/12 [==============================] - 1s 86ms/step - loss: 0.7429 - accuracy: 0.5684 - val_loss: 0.7452 - val_accuracy: 0.5762
Epoch 7/40
12/12 [==============================] - 1s 86ms/step - loss: 0.7410 - accuracy: 0.5684 - val_loss: 0.7433 - val_accuracy: 0.5762
Epoch 8/40
12